In [ ]:
import re
# 注意修改初始基因和染色体数量
def process_data(input_file_path, output_file_path):
    # 打开输入文件和输出文件
    with open(input_file_path, 'r') as infile, open(output_file_path, 'w') as outfile:
        gene = None
        g1 = 97502  # 原始gff的第一个基因
        genes = [g1 + i for i in range(10)]  # 物种染色体的数量

        for line in infile:
            line = line.strip()
            if line.startswith('#'):
                continue
            columns = line.split()

            if 'CDS' in columns[2]:
                attributes = [attr.strip() for attr in columns[8].split(';')]
                gene_id = next((attr.split('=')[1] for attr in attributes if attr.startswith('ID=')), None)
                gene_id = re.sub(r'cds-', '', gene_id)

                if gene == gene_id:
                    continue
                else:
                    gene = gene_id

                columns[0] = re.sub(r'\.1', '', columns[0])  # 删除原始gff第一列的 .1
                columns[0] = re.sub(r'\.2', '', columns[0])  # 删除原始gff第一列的 .2
                columns[0] = re.sub(r'CP0', '', columns[0])  # 删除原始gff第一列多余的字母

                for i, gene_num in enumerate(genes, start=1):
                    columns[0] = re.sub(re.escape(str(gene_num)), str(i), columns[0])

                if columns[0].startswith('JA'):
                    continue  # 删除多余的gff数据

                # 写入到输出文件
                outfile.write('\t'.join(columns) + '\n')

input_file_path1 = "GCA_023547065.1_ASM2354706v1_genomic.gff"  # 请替换为GFF文件的实际路径
output_file_path1 = "test.gff"  # 输出文件的名称

process_data(input_file_path1, output_file_path1)

In [2]:
import os

def process_gff(input_file_path, output_file_path):
    with open(input_file_path, 'r') as file, open(output_file_path, 'w') as output_file:
        for line in file:
            if line.startswith('#'):  # 跳过注释行
                continue
            columns = line.strip().split('\t')
            if columns[2] == 'CDS':  # 检查第三列是否为CDS
                seqid = columns[0]
                start = columns[3]
                end = columns[4]
                score = columns[6]
                # 提取第九列的属性，并处理ID值
                attributes = columns[8].split(';')
                for attribute in attributes:
                    if attribute.startswith('ID='):
                        id_value = attribute.split('-')[1].split(';')[0]  # 提取ID值，并去除"ID="和分号
                        break

                # if id_value.startswith('KA'):
                output_file.write('\t'.join([seqid, start, end, score, id_value]) + '\n')
    os.remove(input_file_path)

# 假设你的GFF文件名为"input.gff"，输出文件名为"output.gff"
input_file_path2 = "test.gff"
output_file_path2 = "test1.gff"

process_gff(input_file_path2, output_file_path2)

In [3]:
insat_file3 = r"test1.gff"
data = []
num = 0
with open(insat_file3) as object_file:
    lines = object_file.readlines()
    for line in lines:
        data.append(line.split())
    for k in range(len(data)):
        bb = data[k][0]
        if data[k][0] != data[k - 1][0]:
            num = 1
            aa = 'Mtr' + bb + 'g' + '%04d' % num
        else:
            num += 1
            aa = 'Mtr' + bb + 'g' + '%04d' % num
        outsat_file = r'Mtr.new.gff'
        with open(outsat_file, 'a') as obj:
            obj.write(bb + '\t' + aa + '\t' + data[k][1] + '\t' + data[k][2] + '\t' + data[k][3] + '\t' + str(num) + '\t' + data[k][4] + '\n')

import os
os.remove(insat_file3)

用于生成lens文件

In [4]:
##以重新命名排序的gff文件为准获取lens文件
ipf = r'Mtr.new.gff'
data = []
with open(ipf) as tj:
    lines = tj.readlines()
    for line in lines:
        data.append(line.split())
    for i in range(len(data)-1):
        if data[i + 1][0] != data[i][0]:
            opt = r'Mtr.lens'
            with open(opt, 'a') as tjo:
                tjo.write(data[i][0] + '\t' + data[i][3] + '\t' + data[i][5] + '\n')
        else:
            pass
    opt = r'Mtr.lens'
    with open(opt, 'a') as tjo:
        tjo.write(data[-1][0] + '\t' + data[-1][3] + '\t' + data[-1][5] + '\n')  

生成cds文件

In [5]:
fr = open(r'GCA_023547065.1_ASM2354706v1_cds_from_genomic.fna')  #读文件
fw = open(r'1.cds', 'w')  #写文件
seq = {}
for line in fr:
    if line.startswith('>'):    #判断字符串是否以‘>开始’
        name = line.split()[0]#.replace('1_cds_','')    #以空格为分隔符
        name = '>' + name.split('_')[2]
        seq[name] = ''
    else:
        seq[name] += line.replace('\n', '')
fr.close()

for i in seq.keys():
   fw.write(i)
   fw.write('\n')
   while len(seq[i]) > 80:
       fw.write(seq[i][:80] + "\n")
       seq[i] = seq[i][80:]

   else:
       fw.write(seq[i] + '\n')
fr.close()
fw.close()


In [6]:
def replace_seq_name(cds_file_path, gff_file_path, output_file_path):
    # 读取GFF文件，创建一个字典来存储第七列和第二列的对应关系
    gff_dict = {}
    with open(gff_file_path, 'r') as gff_file:
        for line in gff_file:
            if line.startswith('#'):  # 跳过注释行
                continue
            columns = line.strip().split('\t')
            gff_dict[columns[6]] = columns[1]  # 存储第七列和第二列的对应关系

    # 读取CDS文件，并替换序列名
    with open(cds_file_path, 'r') as cds_file, open(output_file_path, 'w') as output_file:
        for line in cds_file:
            if line.startswith('>'):  # 匹配序列名行
                seq_name = line.strip()[1:]  # 移除'>'字符
                if seq_name in gff_dict:  # 如果序列名在GFF字典中
                    output_file.write(f">{gff_dict[seq_name]}\n")  # 替换为GFF文件中的第二列内容
                else:
                    output_file.write(line)  # 如果不在字典中，保留原样
            else:
                output_file.write(line)  # 序列数据直接写入输出文件

# 输入文件和输出文件的路径
cds_file_path = "1.cds"  # 请替换为CDS文件的实际路径
gff_file_path = "Mtr.new.gff"  # 请替换为GFF文件的实际路径
output_file_path = "Mtr.cds"  # 输出文件的名称

replace_seq_name(cds_file_path, gff_file_path, output_file_path)
import os
os.remove(cds_file_path)

生成pep文件

In [9]:
def replace_seq_name(cds_file_path, gff_file_path, output_file_path):
    # 读取GFF文件，创建一个字典来存储第七列和第二列的对应关系
    gff_dict = {}
    with open(gff_file_path, 'r') as gff_file:
        for line in gff_file:
            if line.startswith('#'):  # 跳过注释行
                continue
            columns = line.strip().split('\t')
            gff_dict[columns[6]] = columns[1]  # 存储第七列和第二列的对应关系

    # 读取CDS文件，并替换序列名
    with open(cds_file_path, 'r') as cds_file, open(output_file_path, 'w') as output_file:
        for line in cds_file:
            if line.startswith('>'):  # 匹配序列名行
                name = line.split(' ')[0]
                seq_name = name.strip()[1:]  # 移除'>'字符
                if seq_name in gff_dict:  # 如果序列名在GFF字典中
                    output_file.write(f">{gff_dict[seq_name]}\n")  # 替换为GFF文件中的第二列内容
                else:
                    output_file.write(line)  # 如果不在字典中，保留原样
            else:
                output_file.write(line)  # 序列数据直接写入输出文件

# 输入文件和输出文件的路径
cds_file_path = "GCA_023547065.1_ASM2354706v1_protein.faa"  # 请替换为CDS文件的实际路径
gff_file_path = "Mtr.new.gff"  # 请替换为GFF文件的实际路径
output_file_path = "Mtr.pep"  # 输出文件的名称

replace_seq_name(cds_file_path, gff_file_path, output_file_path)

FileNotFoundError: [Errno 2] No such file or directory: 'GCA_026586065.1_Dzin_1.0_protein.faa.faa'